In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import functools
import numpy as np
import pandas as pd

from scipy.special import expit

import sklearn as sk
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf

def split_companies_train_dev_test(companies):
    "Return train, dev, test set for companies"
    train, test = train_test_split(companies, test_size=0.1, stratify = companies.sector)
    train, dev = train_test_split(train, test_size=0.1, stratify = train.sector)
    return train, dev, test


def filter_stocks(stocks, tickers):
    return stocks.loc[tickers]


def df_to_ts(df):
    res = df.copy()
    res.index = pd.DatetimeIndex(pd.to_datetime(res.date))
    res.drop('date', axis=1)
    return res


def log_softmax(x):
    return x - np.log(np.sum(np.exp(x)))


def sigmoid(x):
    return expit(x)


def sample_correlation(df, window_size=63):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    fmap = lambda s: ts['pct_return'].corr(ts[s])
    indices = ts.columns.tolist()[1:]
    correlations = np.array(list(map(fmap, indices)))
    return correlations


def create_correlation_score(df, sample_size=1):
    res = np.array([log_softmax(sample_correlation(df)/0.05)
                    for i in range(sample_size)])
    return np.exp(np.nanmean(res, 0))


def load_data(stock_filename=None, indices_filename=None):

    if stock_filename is None:
        stock_filename = '../../data/processed/wiki_stocks_returns.csv'

    if indices_filename is None:
        indices_filename = '../../data/processed/wiki_indices_returns.csv'

    stocks = pd.read_csv(stock_filename, index_col=False) # long format
    indices = pd.read_csv(indices_filename, index_col=False) # wide format

    # Implementation of hierarchical clustering
    drop_column = lambda df,i=0: df.drop(df.columns[i], axis=1)

    stocks = drop_column(stocks)
    stocks = stocks.drop('name', axis=1)
    stocks = stocks.dropna()

    companies = stocks.groupby('ticker').first().reset_index()
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()

    stocks = stocks.set_index('ticker')

    indices_ts = df_to_ts(indices[['date'] + sectors_unique])
    stocks_ts = df_to_ts(stocks.reset_index())

    stocks_all = pd.merge(stocks_ts, indices_ts, 'left')
    stocks_all = stocks_all.dropna() # loss of 200 000 observations
    stocks_all = stocks_all.drop('sector', axis=1)
    stocks_all = stocks_all.groupby('ticker').apply(df_to_ts)
    stocks_all = stocks_all.drop(['ticker', 'date'], axis=1)
    stocks_all = stocks_all.rename(columns={'close': 'pct_return'})

    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(sectors_counts.index.tolist())
    ticker_to_sector = dict(zip(companies.ticker, label_encoder.transform(companies.sector)))

    return stocks_all, companies, label_encoder, ticker_to_sector

def sectors_statistics(companies):
    sectors_counts = companies.sector.value_counts()
    sectors_proportions = sectors_counts/sectors_counts.sum()
    sectors_unique = sectors_counts.index.tolist()
    return sectors_counts, sectors_proportions, sectors_unique


def add_common_layers(y):
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.LeakyReLU()(y)
    return y


def grouped_convolution(y, nb_channels, _strides, cardinality=4):
    # when `cardinality` == 1 this is just a standard convolution
    if cardinality == 1:
        return keras.layers.Conv1D(nb_channels, kernel_size=10, strides=_strides, padding='same')(y)

    assert not nb_channels % cardinality
    _d = nb_channels // cardinality

    # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
    # and convolutions are separately performed within each group
    groups = []
    for j in range(cardinality):
        group = keras.layers.Lambda(lambda z: z[:, :, j * _d:j * _d + _d])(y)
        groups.append(keras.layers.Conv1D(_d, kernel_size=10, strides=_strides, padding='same')(group))

    # the grouped convolutional layer concatenates them as the outputs of the layer
    y = keras.layers.concatenate(groups)

    return y


def residual_block(y, nb_channels_in, nb_channels_out, cardinality=4, _strides=1, _project_shortcut=False):
    """
    Our network consists of a stack of residual blocks. These blocks have the same topology,
    and are subject to two simple rules:
    - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
    - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
    """
    shortcut = y
    kl = keras.layers
    # we modify the residual building block as a bottleneck design to make the network more economical
    y = kl.Conv1D(nb_channels_in, kernel_size=1, strides=1, padding='same')(y)
    y = add_common_layers(y)

    # ResNeXt (identical to ResNet when `cardinality` == 1)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)

    y = kl.Conv1D(nb_channels_out, kernel_size=1, strides=1, padding='same')(y)
    # batch normalization is employed after aggregating the transformations and before adding to the shortcut
    y = kl.BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != 1:
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = kl.Conv1D(nb_channels_out, kernel_size=1, strides=_strides, padding='same')(shortcut)
        shortcut = kl.BatchNormalization()(shortcut)

    y = kl.add([shortcut, y])

    # relu is performed right after each batch normalization,
    # expect for the output of the block where relu is performed after the adding to the shortcut
    y = kl.LeakyReLU()(y)

    return y



# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = keras.backend.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CovarianceLayer(keras.layers.Layer):

    def __init__(self, num_classes, **kwargs):
        self.num_classes = num_classes
        super(CovarianceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(CovarianceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        series_input, environment_input = inputs
        series_input_multiple = tf.tile(series_input, [1, 1, self.num_classes])
        covariances = tf.reduce_mean(series_input_multiple * environment_input, axis=1)
        
        return covariances

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


def random_subset(df, window_size=21):
    idx = np.random.randint(0, df.shape[0]-window_size)
    ts = df[idx:idx+window_size]
    return ts


def make_keras_subset(dataset_type, companies_data, stocks_data, label_encoder, batch_size, window_size=21):
    idx = np.random.choice(companies_data[dataset_type].shape[0], batch_size)
    df = companies_data[dataset_type].iloc[idx]

    model_input_data = [random_subset(stocks_data[dataset_type].loc[t], window_size) for t in df.ticker]
    model_series_input = np.array([df['pct_return'].values for df in model_input_data])
    model_series_input = model_series_input.reshape(-1, window_size, 1)

    model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])

    y_true = label_encoder.transform(df.sector)

    return model_series_input, model_environment_input, y_true


class StocksSequence(keras.utils.Sequence):

    def __init__(self, stocks_data,  companies_data, window_size, label_encoder, batch_size):
        self.stocks_data = stocks_data
        self.batch_size = batch_size
        self.label_encoder = label_encoder
        self.companies_data = companies_data
        self.window_size = window_size

    def __len__(self):
        return int(np.ceil(self.stocks_data.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):

        idx = np.random.choice(self.companies_data.shape[0], self.batch_size)
        df = self.companies_data.iloc[idx]
        model_input_data = [random_subset(self.stocks_data.loc[t], self.window_size) for t in df.ticker]
        model_series_input = np.array([df['pct_return'].values for df in model_input_data])
        model_series_input = model_series_input.reshape(-1, self.window_size, 1)
        model_environment_input = np.array([df.iloc[:, 1:].values for df in model_input_data])
        y_true = self.label_encoder.transform(df.sector)

        return [model_series_input, model_environment_input], y_true





In [3]:
def make_covariance_convolution_weight(shape, dtype=None):
    kernel_shape = shape[0]
    num_classes = shape[1]
    filters = shape[2]
    x = np.zeros([kernel_shape, num_classes, filters])
    noise = np.random.normal(0, 0.001, size=[kernel_shape, num_classes, filters])

    for j in range(0, filters):
        j_class = j % num_classes
        x[:, [0, j_class], j] = 1
    return x + noise

In [48]:
def make_model(num_classes=16, window_size=21, latent_dim=32):
    kl = keras.layers
    
    series_input = keras.layers.Input(shape=(window_size, 1), dtype='float32', name='series_input')
    environment_input = kl.Input(shape=(window_size, num_classes), dtype='float32', name='environment_input')
    x = kl.Concatenate()([series_input, environment_input])
    x = kl.GaussianNoise(0.001)(x)  
    x = kl.AveragePooling1D(2,1)(x)
    tanh_layer = kl.Lambda(lambda y: tf.tanh(tf.scalar_mul(3, y))) 
    x = tanh_layer(x)    
    x = kl.Conv1D(32, 5, 1, activation='relu', 
                  kernel_initializer=keras.initializers.Orthogonal())(x)  
    x = kl.Conv1D(64, 5, 3, activation='relu',
                  kernel_initializer=keras.initializers.Orthogonal())(x)
    x = kl.Conv1D(128, 3, 2, activation='relu',
                  kernel_initializer=keras.initializers.Orthogonal()))(x)
    # x = kl.Conv1D(32, 5, 2, activation='relu')(x)
    x = kl.Flatten(name='Embedding')(x)
    x_pred = kl.Dense(num_classes, 'sigmoid')(x)

    model = keras.Model(inputs = [series_input, environment_input], outputs=[x_pred], name='Classifier')

    return model

In [8]:
# Make train dev test set.
np.random.seed(42)

### Feature engineering

stock_filename = '../data/processed/wiki_stocks_returns.csv'
indices_filename = '../data/processed/wiki_indices_returns.csv'

stocks_all, companies, label_encoder, ticker_to_sector = load_data(stock_filename, indices_filename)
sectors_counts, sectors_proportions, sectors_unique = sectors_statistics(companies)

max_proportion_baseline = sectors_proportions.max()
biggest_sector = sectors_proportions.argmax()

print("Most representated class:", biggest_sector, ', with proportion of ', round(100*max_proportion_baseline, 2), '%.')
# Accuracy of our models should be better than max_proportion_baseline.

companies_data = {}
data_split = split_companies_train_dev_test(companies)
for i, k in enumerate(['train', 'dev', 'test']):
    companies_data[k] = data_split[i]
stocks_data = {k: filter_stocks(stocks_all, v.ticker) for k, v in companies_data.items()}


Most representated class: Financial Services , with proportion of  13.09 %.


In [51]:
window_size = 63
model = make_model(window_size=window_size)

In [10]:
if False:
    model.load_weights('checkpoint/model_weights_third.json')

In [11]:
import math
def schedule_fn(epoch):
   initial_lrate = 0.001
   drop = 0.8
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

In [52]:
batch_size = 256

optimizer = keras.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

callbacks = [
    keras.callbacks.ModelCheckpoint('checkpoint/model_weights_fourth.json', monitor='val_acc', verbose=1, save_best_only=True, mode='max'),
    keras.callbacks.TensorBoard(log_dir='./logs/twelveth'),
    keras.callbacks.LearningRateScheduler(schedule_fn)
]

stocks_sequence_training = StocksSequence(stocks_data['train'], companies_data['train'], window_size, label_encoder, batch_size)
stocks_sequence_validation = StocksSequence(stocks_data['dev'], companies_data['dev'], window_size, label_encoder, batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
series_input (InputLayer)       (None, 63, 1)        0                                            
__________________________________________________________________________________________________
environment_input (InputLayer)  (None, 63, 16)       0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 63, 17)       0           series_input[0][0]               
                                                                 environment_input[0][0]          
__________________________________________________________________________________________________
gaussian_noise_3 (GaussianNoise (None, 63, 17)       0           concatenate_8[0][0]              
__________

In [53]:
model.fit_generator(
    stocks_sequence_training, steps_per_epoch=1000, epochs=400, callbacks=callbacks, 
    validation_data = stocks_sequence_validation, validation_steps=100, workers=4, max_queue_size=20, verbose=1, 
    use_multiprocessing=True, initial_epoch=0)

stocks_sequence_test = StocksSequence(stocks_data['test'], companies_data['test'], window_size, label_encoder, batch_size)

model.evaluate_generator(stocks_sequence_test, 400, workers=2, use_multiprocessing=True)

Epoch 1/400
1000/1000 [==============================]1000/1000 [==============================] - 116s 116ms/step - loss: 2.5008 - acc: 0.1911 - val_loss: 2.3949 - val_acc: 0.2541

Epoch 2/400
1000/1000 [==============================]1000/1000 [==============================] - 113s 113ms/step - loss: 2.1812 - acc: 0.3126 - val_loss: 2.2170 - val_acc: 0.3222

Epoch 3/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 2.0397 - acc: 0.3607 - val_loss: 2.1107 - val_acc: 0.3521

Epoch 4/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.9808 - acc: 0.3813 - val_loss: 2.1307 - val_acc: 0.3503

Epoch 5/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.9266 - acc: 0.3952 - val_loss: 2.0356 - val_acc: 0.3663

Epoch 6/400
1000/1000 [==============================]1000/1000 [=========================


Epoch 23/400
1000/1000 [==============================]1000/1000 [==============================] - 116s 116ms/step - loss: 1.6317 - acc: 0.4814 - val_loss: 1.8617 - val_acc: 0.4172

Epoch 24/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.6200 - acc: 0.4848 - val_loss: 1.8919 - val_acc: 0.4116

Epoch 25/400
1000/1000 [==============================]1000/1000 [==============================] - 117s 117ms/step - loss: 1.6158 - acc: 0.4872 - val_loss: 1.8617 - val_acc: 0.4142

Epoch 26/400
1000/1000 [==============================]1000/1000 [==============================] - 117s 117ms/step - loss: 1.6165 - acc: 0.4879 - val_loss: 1.8928 - val_acc: 0.4066

Epoch 27/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.6334 - acc: 0.4822 - val_loss: 1.8688 - val_acc: 0.4161

Epoch 28/400
1000/1000 [==============================]1000/1000 [==================


Epoch 46/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.5399 - acc: 0.5122 - val_loss: 1.8381 - val_acc: 0.4292

Epoch 47/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.5397 - acc: 0.5111 - val_loss: 1.8292 - val_acc: 0.4314

Epoch 48/400
1000/1000 [==============================]1000/1000 [==============================] - 116s 116ms/step - loss: 1.5266 - acc: 0.5142 - val_loss: 1.8251 - val_acc: 0.4340

Epoch 49/400
1000/1000 [==============================]1000/1000 [==============================] - 117s 117ms/step - loss: 1.5299 - acc: 0.5141 - val_loss: 1.8338 - val_acc: 0.4297

Epoch 50/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.5178 - acc: 0.5182 - val_loss: 1.7958 - val_acc: 0.4425

Epoch 51/400
1000/1000 [==============================]1000/1000 [==================

1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.4930 - acc: 0.5260 - val_loss: 1.7904 - val_acc: 0.4417

Epoch 70/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.4839 - acc: 0.5286 - val_loss: 1.7970 - val_acc: 0.4483

Epoch 71/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.5111 - acc: 0.5206 - val_loss: 1.8101 - val_acc: 0.4414

Epoch 72/400
1000/1000 [==============================]1000/1000 [==============================] - 116s 116ms/step - loss: 1.4929 - acc: 0.5280 - val_loss: 1.7976 - val_acc: 0.4498

Epoch 73/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.4848 - acc: 0.5286 - val_loss: 1.8180 - val_acc: 0.4386

Epoch 74/400
1000/1000 [==============================]1000/1000 [==============================] 

1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.4562 - acc: 0.5392 - val_loss: 1.7801 - val_acc: 0.4525

Epoch 93/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.4544 - acc: 0.5391 - val_loss: 1.7823 - val_acc: 0.4528

Epoch 94/400
1000/1000 [==============================]1000/1000 [==============================] - 115s 115ms/step - loss: 1.4570 - acc: 0.5407 - val_loss: 1.7875 - val_acc: 0.4497

Epoch 95/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.4570 - acc: 0.5392 - val_loss: 1.7787 - val_acc: 0.4506

Epoch 96/400
1000/1000 [==============================]1000/1000 [==============================] - 114s 114ms/step - loss: 1.4590 - acc: 0.5374 - val_loss: 1.7776 - val_acc: 0.4491

Epoch 97/400
1000/1000 [==============================]1000/1000 [==============================] 

...

Process ForkPoolWorker-6790:
Process ForkPoolWorker-6792:
Process ForkPoolWorker-6735:
Process ForkPoolWorker-6791:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-6793:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-6734:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib

  File "/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py", line 1328, in __getitem__
    return self._getitem_axis(key, axis=0)
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/internals.py", line 3400, in get_slice
    fastpath=True)
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py", line 1695, in _get_slice_axis
    slice_obj = self._convert_slice_indexer(slice_obj, axis)
KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py", line 1749, in _slice
    result = self._constructor(self._data.get_slice(slobj, axis=axis))
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py", line 473, in _set_axis
    self._data.set_axis(axis, labels)
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py", line 1724, in _getitem_axis
    return self._get_slice_axis(key, axis=axis)
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py", line 241, in _convert_slice_indexer
    

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py"

  File "/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py", line 1328, in __getitem__
    return self._getitem_axis(key, axis=0)
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/_impl/keras/utils/data_utils.py", line 402, in get_index
    return _SHARED_SEQUENCES[uid][i]
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/_impl/keras/utils/data_utils.py", line 402, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "<ipython-input-2-fb4e503b1af8>", line 244, in __getitem__
    model_input_data = [random_subset(self.stocks_data.loc[t], self.window_size) for t in df.t

KeyboardInterrupt: 